### 1.문제정의 - 은행 대출 승인 여부

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import numpy as np

### 2.데이터 수집

In [2]:
x_train = pd.read_csv('bank_x_train.csv')
y_train = pd.read_csv('bank_y_train.csv')
x_test = pd.read_csv('bank_x_test.csv')
y_test = pd.read_csv('bank_y_test.csv')

In [3]:
x_train

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,LP001032,Male,No,0,Graduate,No,4950,0.0,125,360,1,Urban
1,1,LP001824,Male,Yes,1,Graduate,No,2882,1843.0,123,480,1,Semiurban
2,2,LP002928,Male,Yes,0,Graduate,No,3000,3416.0,56,180,1,Semiurban
3,3,LP001814,Male,Yes,2,Graduate,No,9703,0.0,112,360,1,Urban
4,4,LP002244,Male,Yes,0,Graduate,No,2333,2417.0,136,360,1,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,LP002585,Male,Yes,0,Graduate,No,3597,2157.0,119,360,0,Rural
380,380,LP001841,Male,No,0,Not Graduate,Yes,2583,2167.0,104,360,1,Rural
381,381,LP002820,Male,Yes,0,Graduate,No,5923,2054.0,211,360,1,Rural
382,382,LP001744,Male,No,0,Graduate,No,2971,2791.0,144,360,1,Semiurban


In [4]:
y_train

,Unnamed: 0,Target
0,0,Y
1,1,Y
2,2,Y
3,3,Y
4,4,Y
...,...,...
379,379,N
380,380,Y
381,381,Y
382,382,Y


In [5]:
del x_train['Unnamed: 0']
del x_test['Unnamed: 0']
del y_train['Unnamed: 0']
del y_test['Unnamed: 0']

In [10]:
x_train = x_train.drop(['Loan_ID'],axis=1) #axis=1 : 축:열방향
x_test = x_test.drop(['Loan_ID'],axis=1)

In [35]:
x_train

array([[  0.,   1.,   1., ..., 125., 360.,   1.],
       [  0.,   1.,   0., ..., 123., 480.,   1.],
       [  0.,   1.,   0., ...,  56., 180.,   1.],
       ...,
       [  0.,   1.,   0., ..., 211., 360.,   1.],
       [  0.,   1.,   1., ..., 144., 360.,   1.],
       [  0.,   1.,   0., ..., 255., 360.,   1.]])

In [12]:
y_train

,Target
0,Y
1,Y
2,Y
3,Y
4,Y
...,...
379,N
380,Y
381,Y
382,Y


In [13]:
x_test

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Female,No,0,Not Graduate,No,3400,0,95,360,1,Rural
1,Male,Yes,0,Graduate,No,14583,0,436,360,1,Semiurban
2,Male,Yes,0,Graduate,No,2500,3796,120,360,1,Urban
3,Male,Yes,1,Graduate,No,3988,0,50,240,1,Urban
4,Male,Yes,2,Graduate,No,3859,0,96,360,1,Semiurban
...,...,...,...,...,...,...,...,...,...,...,...
91,Male,Yes,3+,Graduate,No,39999,0,600,180,0,Semiurban
92,Male,Yes,1,Graduate,No,3052,1030,100,360,1,Urban
93,Male,Yes,3+,Graduate,No,5167,3167,200,360,1,Semiurban
94,Male,Yes,2,Not Graduate,No,3987,1411,157,360,1,Rural


In [14]:
y_test

,Target
0,N
1,Y
2,Y
3,Y
4,Y
...,...
91,Y
92,Y
93,Y
94,Y


### 원핫인코딩

In [15]:
transformer = make_column_transformer((OneHotEncoder(), ['Gender', 'Married','Dependents','Education','Self_Employed','Property_Area']), remainder='passthrough')

In [16]:
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Gender', 'Married', 'Dependents',
                                  'Education', 'Self_Employed',
                                  'Property_Area'])])

In [17]:
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [18]:
x_train

array([[  0.,   1.,   1., ..., 125., 360.,   1.],
       [  0.,   1.,   0., ..., 123., 480.,   1.],
       [  0.,   1.,   0., ...,  56., 180.,   1.],
       ...,
       [  0.,   1.,   0., ..., 211., 360.,   1.],
       [  0.,   1.,   1., ..., 144., 360.,   1.],
       [  0.,   1.,   0., ..., 255., 360.,   1.]])

### 레이블 인코딩

In [19]:
le = LabelEncoder()

In [20]:
le.fit(y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [21]:
labels=['비승인','승인']

In [22]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
y_train

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,

### 4.EDA

### 5-1. 모델링(모델학습, 예측, 평가)

In [24]:
model = VotingClassifier(estimators = [('lr', LogisticRegression()),
                                      ('kn',KNeighborsClassifier()),
                                      ('dt',DecisionTreeClassifier()),
                                      #voting='soft'
                                      ])
#voting: 기본값-hard

In [25]:
model.fit(x_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier())])

In [26]:
model.score(x_train, y_train) #정확도 89%

0.8932291666666666

In [27]:
model.score(x_test, y_test) #정확도 71%

0.71875

#### 새로운 값으로 예측

In [32]:
x_test = [['Male', 'Yes', '0', 'Graduate', 'No', 14583, 0, 436, 360, 1, 'Semiurban']]

x_test = pd.DataFrame(x_test, columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'])

x_test = transformer.transform(x_test)

In [33]:
y_predict = model.predict(x_test)

In [34]:
labels[y_predict[0]]

'승인'